In [134]:
# import library
from pinecone import Pinecone, ServerlessSpec
from langchain_upstage import  UpstageEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.documents import Document
from dotenv import load_dotenv
from kiwipiepy import Kiwi
from collections import Counter
from typing import List, Dict, Tuple
import glob
import os
import hashlib
import math
import re


In [135]:
load_dotenv()
api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=api_key)

In [136]:
index_name = "hybrid"
if index_name not in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=4096, 
        metric="dotproduct",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        ) 
    )
    print(f"{index_name} has been successfully created")
else:
    print(f"{index_name} is already exists.")

hybrid is already exists.


In [137]:


class KoreanBM25Encoder:
    def __init__(self, k1: float = 1.5, b: float = 0.75):
        self.kiwi = Kiwi()
        self.k1 = k1
        self.b = b
        self.avg_doc_length = 0
        self.doc_freqs = Counter()
        self.num_docs = 0
        self.vocabulary = set()
        
    def _tokenize(self, text: str) -> List[str]:
        """키위를 사용하여 텍스트를 형태소 분석"""
        tokens = []
        result = self.kiwi.analyze(text)
        for token, pos, _, _ in result[0][0]:
            if pos.startswith('N') or pos.startswith('V') or pos.startswith('MA'):
                tokens.append(f"{token}_{pos}")
        return tokens
    
    def fit(self, documents: List[str]):
        """문서 집합으로부터 BM25 통계 계산"""
        doc_lengths = []
        term_freqs = []
        
        for doc in documents:
            tokens = self._tokenize(doc)
            doc_lengths.append(len(tokens))
            term_freq = Counter(tokens)
            term_freqs.append(term_freq)
            
            self.doc_freqs.update(term_freq.keys())
            self.vocabulary.update(tokens)
        
        self.num_docs = len(documents)
        self.avg_doc_length = sum(doc_lengths) / self.num_docs if self.num_docs > 0 else 0
        
    def encode_sparse(self, text: str) -> Tuple[List[int], List[float]]:
        """텍스트를 sparse vector로 인코딩 - 리스트 형태로 반환"""
        tokens = self._tokenize(text)
        term_freq = Counter(tokens)
        
        indices = []
        values = []
        
        doc_length = len(tokens)
        
        for term, freq in term_freq.items():
            if term in self.vocabulary:
                idf = math.log((self.num_docs - self.doc_freqs[term] + 0.5) /
                             (self.doc_freqs[term] + 0.5) + 1.0)
                
                numerator = freq * (self.k1 + 1)
                denominator = freq + self.k1 * (1 - self.b + self.b * doc_length / self.avg_doc_length)
                
                bm25_score = idf * (numerator / denominator)
                
                index = abs(hash(term)) % (10 ** 9)
                indices.append(index)
                values.append(float(bm25_score))
        
        return indices, values

def remove_question_sentences(paragraph):
    """불필요한 문장 제거"""
    result = re.sub(r'[^.?!]*[\?]|[^.?!]*보자\.', '', paragraph)
    result = re.sub(r'\s+', ' ', result).strip()
    return result

In [138]:

class HybridSearch:
    def __init__(self, index_name: str, api_key: str):
        """Pinecone hybrid search 초기화"""
        pc = Pinecone(api_key=api_key)
        self.index = pc.Index(index_name)
        self.bm25_encoder = KoreanBM25Encoder()
        self.embeddings = UpstageEmbeddings(model="embedding-passage")
        self.embeddings_query =  UpstageEmbeddings(model="embedding-query")
        
    def index_documents(self, documents: List[Document]):
        """Document 객체 처리 및 인덱싱"""
        # 문서 전처리
        processed_documents = []
        processed_texts = []
        
        for doc in documents:
            processed_text = remove_question_sentences(doc.page_content)
            if processed_text.strip():
                processed_documents.append(doc)
                processed_texts.append(processed_text)
                
        if not processed_documents:
            return
        
        # Dense vectors 생성
        dense_vectors = self.embeddings.embed_documents(processed_texts)
        
        # BM25 encoder 학습
        self.bm25_encoder.fit(processed_texts)
        
        # 각 문서를 인덱싱
        vectors_to_upsert = []
        for doc, text, dense_vec in zip(processed_documents, processed_texts, dense_vectors):
            indices, values = self.bm25_encoder.encode_sparse(text)
            if not indices: continue # upsert 하지 않음.
            # 메타데이터 준비
            metadata = {
                'page': doc.metadata.get('page', ''),
                'source': doc.metadata.get('source', ''),
                # 'subject': doc.metadata.get('subject', ''),
                'text': text
            }
            
            vectors_to_upsert.append({
                'id': hashlib.md5(text.encode()).hexdigest(),
                'values': dense_vec,
                'sparse_values': {
                    'indices': indices,
                    'values': values
                },
                'metadata': metadata
            })
            
            # 배치 처리
            if len(vectors_to_upsert) >= 100:
                self.index.upsert(vectors=vectors_to_upsert)
                vectors_to_upsert = []
        
        # 남은 문서들 처리
        if vectors_to_upsert:
            self.index.upsert(vectors=vectors_to_upsert)
    
    def search(self, query: str, alpha: float = 0.5, top_k: int = 5) -> List[Dict]:
        """하이브리드 검색 수행"""
        dense_vector = self.embeddings_query.embed_query(query)
        indices, values = self.bm25_encoder.encode_sparse(query)
        
        if alpha < 0 or alpha > 1:
            raise ValueError("Alpha must be between 0 and 1")
        hs = {
            'indices': indices,
            'values':  [v * (1 - alpha) for v in values]
        }
        print(hs)
        dense_vector = [v * alpha for v in dense_vector]
        
        results = self.index.query(
            vector=dense_vector,
            sparse_vector= hs,
            top_k=top_k,
            include_metadata=True
        )
        
        return results

In [139]:
INDEX_NAME = "hybrid"
API_KEY = os.environ.get("PINECONE_API_KEY")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150)

split_docs = []
files = sorted(glob.glob("data/*.pdf"))

for file in files:
    loader = PyMuPDFLoader(file)
    split_docs.extend(loader.load_and_split(text_splitter))

# 문서 개수 확인
len(split_docs)
# 샘플 데이터


/opt/homebrew/anaconda3/envs/venv/lib/python3.12/site-packages/langchain_community/document_loaders/parsers/pdf.py:322: UserWarning: Warning: Empty content on page 225 of document data/천재교육_고등교과서_지구과학Ⅱ_오필석(15개정)_교과서 본문.pdf
  warnings.warn(


719

In [140]:
searcher = HybridSearch(INDEX_NAME, API_KEY)

In [141]:
searcher.index_documents(split_docs)

In [142]:
query = "피해 사례 쓰나미 과학 나미와 폭풍 해일에 의 한 피해를 같이"
results = searcher.search(query, alpha=1, top_k=3)

{'indices': [461132265, 863232857, 36473833, 400955895, 462028729, 824163176, 6541746], 'values': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}


In [143]:
results

{'matches': [{'id': 'd8f85d85a8abfb13b34697f1032df000',
              'metadata': {'page': 107.0,
                           'source': 'data/천재교육_고등교과서_지구과학Ⅱ_오필석(15개정)_교과서 '
                                     '본문.pdf',
                           'text': '•해일이 발생하는 여러 가지 원인을 이해할 수 있다. •해일에 의한 피해 '
                                   '사례와 대처 방안을 조사하여 발표할 수 있다. 02 해파는 보통 해수면 위를 '
                                   '부는 바람에 의해서 만들어지지만, 태풍이나 지진에 의해 형성되는 파장이 길고 '
                                   '파고가 높은 해파도 있다. 이러한 해파를 해일이라고 한다. 해 일은 연안으로 '
                                   '접근하면서 높은 파도를 만들어 큰 피해를 주기도 한다. 해일 '
                                   '쓰나미(tsunami) 일본에서는 항구의 파도라는 뜻 으로 큰 해일을 '
                                   '지칭하는 용어이 다. 1946년 알류샨 열도 지진과 함께 발생한 해일로 '
                                   '하와이에 큰 피해가 발생했는데, 일본계 미국 인이 쓰나미라는 용어를 사용하 '
                                   '면서 국제 사회에서 공식 명칭으 로 쓰이기 시작하였다. 스스로 생각해 보기 '
                                   '로 해 그림은 쓰나미가 나타난 해안의 모습이다. 쓰나미가 나타